## Kubernetes Learing


### command


In [ ]:
#设置污点
kubectl taint nodes node1 key=value:effect

# 去除污点
kubectl taint nodes node1 key:effect-

# 去除所有污点
kubectl taint nodes node1 key-

kubectl taint nodes node1 tag=heima:PreferNoSchedule
kubectl taint nodes node1 tag=heima:NoSchedule
kubectl taint nodes node1 tag=heima:NoExecute 